In [25]:
import pandas as pd
import requests

In [29]:
url = 'https://www.alphavantage.co/query?function=DIGITAL_CURRENCY_DAILY&symbol=BTC&market=CNY&apikey=demo&datatype=csv'
response = requests.get(url)
print(response)

<Response [200]>


In [30]:
with open('data.csv', 'wb') as dataFile:
    dataFile.write(response.content)

In [31]:
data = pd.read_csv('data.csv')
data

,timestamp,open (CNY),high (CNY),low (CNY),close (CNY),open (USD),high (USD),low (USD),close (USD),volume,market cap (USD)
0,2023-05-01,202016.028600,202615.860000,201896.684265,202428.861870,29233.20,29320.00,29215.93,29292.94,1281.336740,1281.336740
1,2023-04-30,201997.024725,207103.469595,200954.506695,202016.097705,29230.45,29969.39,29079.59,29233.21,39752.537200,39752.537200
2,2023-04-29,202558.433745,203506.485240,200618.725500,201997.024725,29311.69,29448.88,29031.00,29230.45,20466.830580,20466.830580
3,2023-04-28,203671.577085,204547.621170,199651.255500,202558.502850,29472.77,29599.54,28891.00,29311.70,54298.165780,54298.165780
4,2023-04-27,196363.861545,206554.845000,196112.112030,203671.577085,28415.29,29890.00,28378.86,29472.77,95430.824310,95430.824310
...,...,...,...,...,...,...,...,...,...,...,...
995,2020-08-09,81274.528710,81523.928655,79622.573685,80726.249640,11761.02,11797.11,11521.97,11681.68,41493.067342,41493.067342
996,2020-08-08,80122.824780,81601.049835,79553.676000,81277.223805,11594.36,11808.27,11512.00,11761.41,41858.161040,41858.161040
997,2020-08-07,81284.548935,82303.640370,78240.681000,80121.926415,11762.47,11909.94,11322.00,11594.23,65755.926022,65755.926022
998,2020-08-06,81163.200555,82234.950000,79902.656250,81284.479830,11744.91,11900.00,11562.50,11762.46,63529.085020,63529.085020
